In [1]:
import tensorflow as tf
from pathlib import Path

In [2]:
import os
os.chdir('../')

In [3]:
os.getcwd()

'c:\\DS_Projects\\DeepCNN_Classifier_End_To_End'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/vsuraj25/DeepCNN_Classifier_End_To_End.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "vsuraj25"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7cc9630c696040acc98fd0620e6dfd29cb54c77d"

In [5]:
from dataclasses import dataclass

In [6]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    path : Path
    training_data : Path
    params_img_size : list
    params_batch_size : int
    mlflow_uri : str
    all_params : dict
    

In [7]:
from DeepCNNClassifer.constants import *
from DeepCNNClassifer.utils import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(
            path = 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/PetImages',
            mlflow_uri = 'https://dagshub.com/vsuraj25/DeepCNN_Classifier_End_To_End.mlflow',
            params_img_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE,
            all_params = self.params
        )
        return model_evaluation_config

In [9]:
from DeepCNNClassifer.utils import *
import mlflow 
import mlflow.keras
from urllib.parse import urlparse

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
                target_size = self.config.params_img_size[:-1],
                batch_size = self.config.params_batch_size,
                interpolation = "bilinear"
            )
        
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path : Path):
        return tf.keras.models.load_model(path)

    def evaluate(self):
        self.model = self.load_model(self.config.path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_scores(self):
        scores = {
            "loss" : self.score[0],
            "accuracy" : self.score[1]
        }
        return save_json(path = Path('scores.json'), data = scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss" : self.score[0],
                "accuracy" : self.score[1]
            })
            if tracking_uri_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name = "VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [11]:
try:
    config = ConfigurationManager()
    validation_config = config.get_evaluation_config()
    evaluation = ModelEvaluation(validation_config)
    evaluation.evaluate()
    evaluation.save_scores()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 85s 153ms/step - loss: 1.0353 - accuracy: 0.8986


2023/05/13 11:44:02 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\sv255\AppData\Local\Temp\tmp3rexvsfm\model\data\model\assets


c:\DS_Projects\DeepCNN_Classifier_End_To_End\env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2023/05/13 11:44:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
